In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.insert(0, '..')
import fashion.preprocessing as prep
# display all columns
pd.set_option('display.max_columns', None)

# show all results of the notebook not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import networkx as nx
from node2vec import Node2Vec

In [ ]:
shops = prep.load_shops('../data/20200120_filiali.csv', extra_info=False)
shops.head()


In [ ]:
#This code takes an hour or so to run

from tqdm.notebook import tqdm

import pgeocode
dist = pgeocode.GeoDistance('it')

dist.query_postal_code("95024", "80021")


adjacency_list = []

for x_index, x in tqdm(shops.iterrows()):
    for y_index, y in (shops.iterrows()):
        if y_index <= x_index:
            pass
        else:
            z_weight = dist.query_postal_code(x.ZipCode, y.ZipCode)
            
            adjacency_list.append([x_index,y_index,z_weight])
            
data_transposed = zip(adjacency_list)

columns = ['node1','node2','weight']

adjacency_frame = pd.DataFrame(adjacency_list, columns = columns)


adjacency_frame.to_csv(r'store2store_graph.csv', header=False, index=False)




In [2]:
# Create a graph

G = nx.read_weighted_edgelist("store2store_graph.csv", delimiter=',')


# There's probably an error caused by those empty rows you got rid of in the 'clean' version of this!

TypeError: Failed to convert weight data  to type <class 'float'>.

In [ ]:
# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(G, dimensions=10, walk_length=30, num_walks=100, workers=1)  # Use temp_folder for big graphs

model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)


model.wv.save_word2vec_format(r'store2store_graph.emb')

In [ ]:
# Create a graph

fh=open("store2store_graph.csv", 'r')
graph = nx.read_weighted_edgelist(fh, delimiter=',')


# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(graph, dimensions=10, walk_length=30, num_walks=100, workers=1)  # Use temp_folder for big graphs

model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)


model.wv.save_word2vec_format(r'store2store_graph.emb')